## Endpoint:

```python
#retomamos la variable sp de arriba que recordamos que es el acceso a la API con el TOKEN:

sp.search(q, limit=10, offset=0, type='track', market=None)

```
`search`: searches for an item

*Parameters*:

`q`: the search query (see how to write a query in the official documentation https://developer.spotify.com/documentation/web-api/reference/search/) #noqa

`limit`: the number of items to return (min = 1, default = 10, max = 50). The limit is applied *within each type*, not on the total response.

`offset`: the index of the first item to return.
Default: offset = 0
Range: 0 - 1000
Example: offset=5


`type`: the types of items to return. One or more of ‘artist’, ‘album’, ‘track’, ‘playlist’, ‘show’, and ‘episode’. If multiple types are desired, pass in a comma separated string; e.g., ‘track,album,episode’.

`market`:  An ISO 3166-1 alpha-2 country code or the string from_token


In [1]:
#pip install spotipy

In [312]:
import requests

import re

import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# Coloca tus credenciales aquí
CLIENT_ID = '90527396d8f84493aae641a416e24f17'
CLIENT_SECRET = '0d68e0bcec03408890499dcb774bd3f6'

# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

## Creación de un filtro:

He creado un filtro para saber si el género es fiable o no. Por supuesto, este filtro puede ser inexacto, pero sobre todo, sirve para quitar morralla o cosas que no son válidas, sobre todo de cara al género pop.

In [338]:
genero_seleccionado = 'pop'
genero = sp.search(q = f"genre:{genero_seleccionado} year:2021", limit=50, offset=50, type = "artist,track", market='ES') #hago la petición

In [339]:
def verificar_genero(genero):
    """ 
    función para comprobar los datos del genero de los artistas
    """
#Creamos diciconario vacío que vamos a rellenar con la iteracion:

    gen2021 = {
        'género': [],
        'artista': []
        }


#Iteramos para rellenar el diccionario:
    for element in genero['artists']['items']:
        gen2021['género'].append(element['genres'])


    for element in genero['tracks']['items']:
        for el in element['artists']:
            gen2021['artista'].append(el['name'])

    return gen2021


In [340]:
verificar_genero(genero)

{'género': [['latin pop'],
  [],
  ['latin pop'],
  ['flamenco', 'flamenco urbano', 'flamenco pop'],
  ['latin pop'],
  [],
  ['k-pop'],
  ['edm'],
  [],
  [],
  [],
  ['rap', 'hip hop'],
  ['latin pop'],
  ['latin pop'],
  [],
  ['flamenco', 'flamenco pop', 'flamenco urbano'],
  ['flamenco', 'flamenco urbano', 'flamenco pop'],
  [],
  [],
  ['edm'],
  [],
  ['k-pop'],
  ['pop'],
  [],
  ['flamenco', 'flamenco pop'],
  [],
  ['edm', 'trance', 'big room', 'house', 'dance'],
  [],
  ['salsa', 'salsa romantica', 'merengue', 'bachata', 'latin', 'latin pop'],
  ['reggaeton'],
  ['reggaeton', 'latin'],
  [],
  [],
  [],
  ['soft pop'],
  ['indie'],
  [],
  ['soft pop'],
  ['trova', 'singer-songwriter', 'nueva trova'],
  ['bachata', 'bachata pop'],
  [],
  ['hyperpop', 'art pop'],
  ['latin pop', 'latin'],
  ['brazilian pop', 'funk pop', 'funk carioca', 'funk', 'pagode baiano'],
  ['flamenco', 'flamenco pop', 'flamenco urbano'],
  ['flamenco pop', 'flamenco'],
  ['bachata', 'latin', 'bachata 

## Retomando la función de Sara con los comentarios de Isa:

#### Actualizaciones:

- Actualización del Docstring (documentación de la función) incluyendo la fecha ordenando los parámetros. Me parece que el genero no tenía el parámetrro por defecto de “pop”, aunque en la documentación Sara sí lo había reflejado. ¿Queréis que lo incluyamos?

- He hecho un cambio añadiendo un format en la llamada a la API=== (f”{ }” ) para incluir en la llamada a la API el año (como decía Nuria en su comentario de Slack!)

- He añadido en la función un filtro de género, porque he hecho pruebas y los géneros solicitados no coincidían con los géneros que nos devolvían. Sobre todo con “pop”. En este género, nos devuelve prácticamente los mismos datos que lo que sacamos en reggaeton. En cambio, el indie se categoriza muy bien y a ese no le he hecho pasar el filtro. Para hacer esto, he añadido en la llamada a la API, en “type” a “artista” además del track, porque el género musical viene definido en el artista y no en la canción… 😭.

In [359]:
def rastreo_global(genero, año, limit=50, market='ES', max_results=200):
    """rastreo_global:Realiza una búsqueda global de canciones de un género en Spotify y con un año determinado.

    Arguments:
        genero {str} -- Género musical a buscar
        año (int o str) -- Año específico para filtrar 

    Keyword Arguments:
        limit {int} -- Número de canciones por solicitud (máx. 50). (default: {50})
        market {str} -- Mercado donde se puede escuchar cada canción (default: {'ES'})
        max_results {int} -- Número total de resultados deseados (default: {200})

    Returns:
        datos {list} -- lista de dicionarios de canciones con información relevante 
    """
    
    datos = []
    offset = 0  # Para la paginación

    while len(datos) < max_results:
        # 1. Realizar la llamada a la API en Spotify
        busqueda = sp.search(q=f"genre:{genero} year:{str(año)}", limit=limit, offset=offset, type='artist,track', market=market)
        items_tracks = busqueda['tracks']['items']
        items_artists = busqueda['artists']['items']
        
        # Si no hay más resultados, salir del bucle
        if not items_tracks:
            break
        

        # 2. FILTRO para comprobar si el género obtenido de la API es el correcto:
        
        ## PARTE 1: definimos el filtro:
        variable_genero = []
        if genero == 'pop':
            variable_genero = ['pop', 'edm', 'dance']
        elif genero == 'rap':
            variable_genero = ['hip', 'hop', 'rap']
        elif genero == 'reggaeton':
            variable_genero = ['reggaeton', 'latin', 'trap']

        # PARTE 2: aplicamos el filtro:
        if genero == 'indie':
            pass #el género indie no lo filtramos porque es muy fiable el que saca de la llamada a la API
            ## en otras palabras, no filtramos indie, pero seguimos con la creación del diccionario
        
        elif genero == 'pop' or genero == 'rap' or genero == 'reggaeton':
            for num, element in enumerate(items_artists):
                if not any (genre in variable_genero for genre in element['genres']):
                    items_tracks[num]['name'] = 'canción no válida'
                    
        else:
            print('AVISO: has insertado un género que no vamos a comprobar si es correcto')

        #3. Creamos el diccionario (parte de Sara e Isa)
        for item in items_tracks:
            track_info = {
                'track_name': item['name'],
                'artist_name': item['artists'][0]['name'],  # Primer artista
                'track_year': año,
                'release_date': item['album']['release_date'],
                'id_track': item['id']
                }
            
            datos.append(track_info)

            # Detenernos si alcanzamos el máximo de resultados
            if len(datos) >= max_results:
                break

        #4. Aumentamos el offset para la siguiente paginación
        offset += limit
    
    return datos

In [360]:
reggaeton= rastreo_global ('reggaeton', 2022) 


In [361]:
reggaeton[0]

{'track_name': 'La Bachata',
 'artist_name': 'Manuel Turizo',
 'track_year': 2022,
 'release_date': '2022-05-26',
 'id_track': '5ww2BF9slyYgNOk37BlC4u'}

In [362]:
df_reggaeton = pd.DataFrame(reggaeton)
print(df_reggaeton)

                  track_name       artist_name  track_year release_date  \
0                 La Bachata     Manuel Turizo        2022   2022-05-26   
1              Ojitos Lindos         Bad Bunny        2022   2022-05-06   
2    Feliz Cumpleaños Ferxxo              Feid        2022   2022-09-14   
3                Moscow Mule         Bad Bunny        2022   2022-05-06   
4                     Normal              Feid        2022   2022-09-14   
..                       ...               ...         ...          ...   
195                Un Tiempo  Cali Y El Dandee        2022   2022-02-25   
196        canción no válida          Anuel AA        2022   2022-12-09   
197                    Nieve              Feid        2022   2022-03-04   
198                    WUEPA    Rauw Alejandro        2022   2022-02-25   
199        canción no válida          Anuel AA        2022   2022-12-09   

                   id_track  
0    5ww2BF9slyYgNOk37BlC4u  
1    3k3NWokhRRkEPhCzPmV8TW  
2    2CeK